In [1]:
#Load the companies and rounds data into 2 data frames - companies and rounds2 respectively | Start
import numpy as np
import pandas as pd
import warnings

# ignore warnings
warnings.filterwarnings('ignore')



companies = pd.read_csv("/Users/achintyadutta/Documents/UPGRAD/DataWarehouse - SQL/Group Asssignment/companies.txt",sep="\t", encoding="ISO-8859-1")
rounds2 = pd.read_csv("/Users/achintyadutta/Documents/UPGRAD/DataWarehouse - SQL/Group Asssignment/rounds2.csv", sep=",",encoding="ISO-8859-1")

#To convert unique id to lower case and remove special characters-companies dataframe
companies.permalink = companies.permalink.str.encode("utf-8").str.decode("ascii","ignore")
companies["permalink"]= companies["permalink"].str.lower()

#To convert unique id to lower case and remove special characters-rounds dataframe
rounds2.company_permalink = rounds2.company_permalink.str.encode("utf-8").str.decode("ascii","ignore")
rounds2.company_permalink = rounds2["company_permalink"].str.lower()


#Load the companies and rounds data into 2 data frames - companies and rounds2 respectively | End

#set option to display float format
pd.set_option('display.float_format', lambda x: '%.3f' % x)


companies.describe()
companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66368 entries, 0 to 66367
Data columns (total 10 columns):
permalink        66368 non-null object
name             66367 non-null object
homepage_url     61310 non-null object
category_list    63220 non-null object
status           66368 non-null object
country_code     59410 non-null object
state_code       57821 non-null object
region           58338 non-null object
city             58340 non-null object
founded_at       51147 non-null object
dtypes: object(10)
memory usage: 5.1+ MB


In [2]:
#How many unique companies are present in rounds2? | Start

rounds2.company_permalink = rounds2.company_permalink.str.encode("utf-8").str.decode("ascii","ignore")
rounds2["company_permalink"].str.lower().nunique()

#How many unique companies are present in rounds2? | End

66368

In [3]:
#Are there any companies in the rounds2 file which are not present in companies? Answer yes or no: Y/N
companies[~companies["permalink"].isin(rounds2["company_permalink"])]

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at


In [4]:
#How many unique companies are present in companies? | Start

companies.permalink = companies.permalink.str.encode("utf-8").str.decode("ascii","ignore")
companies["permalink"].str.lower().nunique()

#How many unique companies are present in companies? | End

66368

In [5]:
#Merge 2 dataframes | Start

master_frame=pd.merge(rounds2,companies,how='inner',left_on=['company_permalink'],right_on=['permalink'])

#Merge 2 dataframes | End


#Compute the NaN values | Start
master_frame=master_frame.loc[~np.isnan(master_frame['raised_amount_usd'])]
#Compute the NaN values | End



In [6]:
#Average funding amount | Start

avg_amt = master_frame.groupby('funding_round_type',as_index=False)['raised_amount_usd'].mean()
filter_avg_amt = avg_amt[avg_amt.funding_round_type.isin(['venture','angel','seed','private_equity'])]
filter_avg_amt

#Average funding amount | End

,funding_round_type,raised_amount_usd
0,angel,958694.470
8,private_equity,73308593.029
11,seed,719817.997
13,venture,11748949.129


In [7]:
#Considering that Spark Funds wants to invest between 5 to 15 million USD per investment round, which investment type is the most suitable for it?

master_frame_segmemtation = master_frame.groupby('funding_round_type')
pd.options.display.float_format = '{:.2f}'.format
master_frame_amount = master_frame_segmemtation['raised_amount_usd'].mean().reset_index().sort_values(by='raised_amount_usd',ascending=False)


master_frame_amount.loc[(master_frame_amount['raised_amount_usd'] >= 5000000) & (master_frame_amount['raised_amount_usd'] <= 15000000) ]

,funding_round_type,raised_amount_usd
13,venture,11748949.13


In [8]:
# top 9 calculation | Start

master_frame_venture = master_frame.loc[(master_frame.funding_round_type == 'venture') ]
master_frame_countrysum=master_frame_venture.groupby('country_code').raised_amount_usd.sum().to_frame().reset_index().rename(columns={'raised_amount_usd':'Total'})
top9=master_frame_countrysum.sort_values(by='Total',ascending=False).head(9)

# top 9 calculation | End

# top 9 English speaking or Non English speaking countries | Start

top9.loc[top9['country_code'] == 'USA','English_Speaking']='YES'
top9.loc[top9['country_code'] == 'CHN','English_Speaking']='NO'
top9.loc[top9['country_code'] == 'GBR','English_Speaking']='YES'
top9.loc[top9['country_code'] == 'IND','English_Speaking']='YES'
top9.loc[top9['country_code'] == 'CAN','English_Speaking']='YES'
top9.loc[top9['country_code'] == 'FRA','English_Speaking']='NO'
top9.loc[top9['country_code'] == 'ISR','English_Speaking']='NO'
top9.loc[top9['country_code'] == 'DEU','English_Speaking']='NO'
top9.loc[top9['country_code'] == 'JPN','English_Speaking']='NO'

top9
# top 9 English speaking or Non English speaking  countries | End



,country_code,Total,English_Speaking
94,USA,422510842796.00,YES
15,CHN,39835418773.00,NO
29,GBR,20245627416.00,YES
39,IND,14391858718.00,YES
12,CAN,9583332317.00,YES
28,FRA,7259536732.00,NO
42,ISR,6907514579.00,NO
21,DEU,6346959822.00,NO
45,JPN,3363676611.00,NO


In [9]:
#Investment in top 3 english speaking countries | Start

top3=top9.loc[top9['English_Speaking']=='YES'].sort_values(by='Total',ascending=False).head(3)

#Investment in top 3 english speaking countries | End
top3

,country_code,Total,English_Speaking
94,USA,422510842796.00,YES
29,GBR,20245627416.00,YES
39,IND,14391858718.00,YES


In [10]:
# Load mapping csv file | Start
mapping_df = pd.read_csv("/Users/achintyadutta/Documents/UPGRAD/DataWarehouse - SQL/Group Asssignment/mapping.csv",sep=",",encoding='ISO-8859-1')
# Load mapping csv file | End


# Extract the primary sector of each category list from the category_list column | Start

catList = master_frame_venture.category_list.str.split('|',n=1,expand=True)
master_frame_venture['primary sector'] = catList[0]

# Extract the primary sector of each category list from the category_list column | End



#Map the category_list with main_sector and store in mapping_t dataframe
mapping_t=pd.melt(mapping_df,id_vars='category_list',var_name='main_sector',value_name='value')
mapping_t.dropna(subset=['category_list'])
mapping_t= mapping_t[mapping_t.value !=0]
mapping_t = mapping_t[['category_list','main_sector']]


In [11]:
# Use the mapping file 'mapping.csv' to map each primary sector to one of the eight main sectors (Note that ‘Others’ is also considered one of the main sectors) | Start

primary_mapping = pd.merge(master_frame_venture,mapping_t,how="inner",left_on=["primary sector"],right_on=["category_list"])

# Use the mapping file 'mapping.csv' to map each primary sector to one of the eight main sectors (Note that ‘Others’ is also considered one of the main sectors) | End


In [12]:
# Remove NaN values from primary_mapping df | Start

primary_mapping = primary_mapping.dropna(subset=['primary sector'])

# Remove NaN values from primary_mapping df | End

In [57]:
#Create three separate data frames D1, D2 and D3 for each of the three countries containing the observations of funding type FT falling within the 5-15 million USD range. The three data frames should contain:

    # 1) All the columns of the master_frame along with the primary sector and the main sector

    # 2) The total number (or count) of investments for each main sector in a separate column

    # 3) The total amount invested in each main sector in a separate column
    
D=primary_mapping.loc[(primary_mapping['raised_amount_usd'] >= 5000000) & (primary_mapping['raised_amount_usd'] <= 15000000)]

#pd.options.mode.chained_assignment = None  # default='warn'
#D.loc[D["Automotive & Sports"] == 1,"main_sector"] = "Automotive & Sports"
#D.loc[D["Cleantech / Semiconductors"] == 1,"main_sector"] = "Cleantech / Semiconductors"
#D.loc[D["Entertainment"] == 1,"main_sector"] = "Entertainment"
#D.loc[D["Health"] == 1,"main_sector"] = "Health"
#D.loc[D["Manufacturing"] == 1,"main_sector"] = "Manufacturing"
#D.loc[D["News, Search and Messaging"] == 1,"main_sector"] = "News, Search and Messaging"
#D.loc[D["Others"] == 1,"main_sector"] = "Others"
#D.loc[D["Social, Finance, Analytics, Advertising"] == 1,"main_sector"] = "Social, Finance, Analytics, Advertising"

#D=D.drop(["Automotive & Sports","Cleantech / Semiconductors","Entertainment","Health","Manufacturing","News, Search and Messaging","Others","Social, Finance, Analytics, Advertising"],axis=1)

# Create D1 | Start

D1 = D.loc[D["country_code"]=='USA']

# Create D1 | End



# Create D2 | Start

D2 = D.loc[D["country_code"]=='GBR']

# Create D2 | End



# Create D3 | Start

D3 = D.loc[D["country_code"]=='IND']

# Create D3 | End


D1

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,permalink,name,homepage_url,category_list_x,status,country_code,state_code,region,city,founded_at,primary sector,category_list_y,main_sector
4,/organization/all-def-digital,/funding-round/452a2342fe720285c3b92e9bd927d9ba,venture,A,06-08-2014,5000000.00,/organization/all-def-digital,All Def Digital,http://alldefdigital.com,Media,operating,USA,CA,Los Angeles,Los Angeles,NaN,Media,Media,Entertainment
16,/organization/chefs-feed,/funding-round/adca195749ae9ace84684723fbe75e5b,venture,A,26-02-2015,5000000.00,/organization/chefs-feed,ChefsFeed,http://www.chefsfeed.com,Media|Mobile|Restaurants|Technology,operating,USA,CA,SF Bay Area,San Francisco,01-01-2012,Media,Media,Entertainment
24,/organization/huffingtonpost,/funding-round/7f05940c4d2dfecb8e50a0e5720e5065,venture,A,01-08-2006,5000000.00,/organization/huffingtonpost,The Huffington Post,http://www.huffingtonpost.com,Media|News|Publishing,acquired,USA,NY,New York City,New York,09-05-2005,Media,Media,Entertainment
25,/organization/huffingtonpost,/funding-round/9241ae16e08df17ebdc064e49e23035a,venture,B,01-09-2007,5000000.00,/organization/huffingtonpost,The Huffington Post,http://www.huffingtonpost.com,Media|News|Publishing,acquired,USA,NY,New York City,New York,09-05-2005,Media,Media,Entertainment
33,/organization/matchmine,/funding-round/41ac526630da57ad6eb9d02431b17657,venture,A,01-09-2007,10000000.00,/organization/matchmine,MatchMine,http://matchmine.com,Media|News|Reviews and Recommendations,closed,USA,MA,Boston,Needham,01-01-2007,Media,Media,Entertainment
35,/organization/mediabong,/funding-round/9282890ca87072025dc1807f400acee6,venture,B,13-05-2015,5000000.00,/organization/mediabong,MEDIABONG,http://www.mediabong.com,Media|Semantic Search|Video,operating,USA,NY,New York City,New York,30-05-2011,Media,Media,Entertainment
38,/organization/newscorporation,/funding-round/8f6d7c4592e43e91e8688ba342bffcb7,venture,NaN,08-01-2010,12500000.00,/organization/newscorporation,News Corp,http://www.newscorp.com,Media|News|Publishing,ipo,USA,NY,New York City,New York,01-01-2013,Media,Media,Entertainment
39,/organization/nokeena,/funding-round/9225f2db6b1b74892d5de6a8744b94ea,venture,A,08-08-2008,9400000.00,/organization/nokeena,Ankeena Networks,http://www.ankeena.com,Media|Software,acquired,USA,CA,SF Bay Area,Santa Clara,01-01-2008,Media,Media,Entertainment
41,/organization/nokeena,/funding-round/fb7a617ed6d1a3203024f0f111417bfb,venture,B,25-06-2009,6500000.00,/organization/nokeena,Ankeena Networks,http://www.ankeena.com,Media|Software,acquired,USA,CA,SF Bay Area,Santa Clara,01-01-2008,Media,Media,Entertainment
45,/organization/plumtv,/funding-round/e5109c28c1b4899b068cfa552850c424,venture,B,01-09-2009,5200000.00,/organization/plumtv,PlumTV,http://www.plumtv.com,Media|Television|Web Hosting,closed,USA,NY,New York City,New York,01-01-2002,Media,Media,Entertainment


In [14]:
#1. Total number of investments (count) - USA | Start

len(D1.index)

#1. Total number of investments (count) - USA | End

11149

In [15]:
#1. Total number of investments (count) - GBR | Start

len(D2.index)

#1. Total number of investments (count) - GBR | End

577

In [16]:
#1. Total number of investments (count) - IND | Start

len(D3.index)

#1. Total number of investments (count) - IND | End

299

In [17]:
# Total investement & Total investment count for USA | Start

D1_count=D1.groupby('main_sector')['main_sector'].count().reset_index(name="count")
D1_count
D1_tot_amt = D1.groupby('main_sector')['raised_amount_usd'].sum().reset_index(name="sum")
D1_tot_amt
D1= pd.merge(pd.merge(D1,D1_count,on='main_sector'),D1_tot_amt,on='main_sector')

# Total investement & Total investment count for USA | End

In [18]:
# Total investment for D1 | Start
D1['raised_amount_usd'].sum()
# Total investment for D1 | End

99661524549.0

In [19]:
#Top Sector name (no. of investment-wise) - D1 | Start
D1_count.sort_values(by='count', ascending=False).iloc[0,:].main_sector
#Top Sector name (no. of investment-wise) - D1 | End

'Others'

In [20]:
#Second Sector name (no. of investment-wise) - D1 | Start
D1_count.sort_values(by='count', ascending=False).iloc[1,:].main_sector
#Second Sector name (no. of investment-wise)) - D1 | End

'Cleantech / Semiconductors'

In [21]:
#Third Sector name (no. of investment-wise)- D1 | Start
D1_count.sort_values(by='count', ascending=False).iloc[2,:].main_sector
#Third Sector name (no. of investment-wise)- D1 | End

'Social, Finance, Analytics, Advertising'

In [22]:
#Number of investments in top sector (3) - D1 | Start
D1[D1.main_sector ==D1_count.sort_values(by='count', ascending=False).iloc[0,:].main_sector].shape
#Number of investments in top sector (3) - D1 | End

(2923, 21)

In [23]:
#Number of investments in second sector (4) - D1 | Start
D1[D1.main_sector ==D1_count.sort_values(by='count', ascending=False).iloc[1,:].main_sector].shape
#Number of investments in second sector (4) - D1 | End

(2297, 21)

In [24]:
#Number of investments in third sector (5) - D1 | Start
D1[D1.main_sector ==D1_count.sort_values(by='count', ascending=False).iloc[2,:].main_sector].shape
#Number of investments in third sector (5) - D1 | End

(1912, 21)

In [25]:
# For point 3 (top sector count-wise), which company received the highest investment | Start

D1[D1.main_sector ==D1_count.sort_values(by='count', ascending=False).iloc[0,:].main_sector].groupby('company_permalink',as_index=False)['raised_amount_usd'].sum().sort_values(by='raised_amount_usd', ascending=False).head(1)

# For point 3 (top sector count-wise), which company received the highest investment | End

,company_permalink,raised_amount_usd
1923,/organization/virtustream,64300000.00


In [26]:
#For point 4 (second best sector count-wise), which company received the highest investment | Start
D1[D1.main_sector ==D1_count.sort_values(by='count', ascending=False).iloc[1,:].main_sector].groupby('company_permalink',as_index=False)['raised_amount_usd'].sum().sort_values(by='raised_amount_usd', ascending=False).head(1)
#For point 4 (second best sector count-wise), which company received the highest investment | End

,company_permalink,raised_amount_usd
243,/organization/biodesix,75300000.00


In [27]:
# Total investment for D2 | Start
D2['raised_amount_usd'].sum()
# Total investment for D2 | End

5028704358.0

In [29]:
# Total investement & Total investment count for GBR - D2 | Start

D2_count=D2.groupby('main_sector')['main_sector'].count().reset_index(name="count")
D2_count
D2_tot_amt = D1.groupby('main_sector')['raised_amount_usd'].sum().reset_index(name="sum")
D2_tot_amt
D2= pd.merge(pd.merge(D2,D2_count,on='main_sector'),D2_tot_amt,on='main_sector')

# Total investement & Total investment count for GBR -  D2| End

In [30]:
#Top Sector name (no. of investment-wise) - D2 | Start
D2_count.sort_values(by='count', ascending=False).iloc[0,:].main_sector
#Top Sector name (no. of investment-wise) - D2 | End

'Others'

In [31]:
#Second Sector name (no. of investment-wise) - D2 | Start
D2_count.sort_values(by='count', ascending=False).iloc[1,:].main_sector
#Second Sector name (no. of investment-wise)) - D2| End

'Cleantech / Semiconductors'

In [33]:
#Third Sector name (no. of investment-wise)- D2 | Start
D2_count.sort_values(by='count', ascending=False).iloc[2,:].main_sector
#Third Sector name (no. of investment-wise)- D2 | End

'Social, Finance, Analytics, Advertising'

In [34]:
#Number of investments in top sector (3) - D2 | Start
D2[D2.main_sector ==D2_count.sort_values(by='count', ascending=False).iloc[0,:].main_sector].shape
#Number of investments in top sector (3) - D2 | End

(143, 21)

In [35]:
#Number of investments in second sector (4) - D2 | Start
D2[D2.main_sector ==D2_count.sort_values(by='count', ascending=False).iloc[1,:].main_sector].shape
#Number of investments in second sector (4) - D2 | End

(127, 21)

In [36]:
#Number of investments in third sector (5) - D2 | Start
D2[D2.main_sector ==D1_count.sort_values(by='count', ascending=False).iloc[2,:].main_sector].shape
#Number of investments in third sector (5) - D2 | End

(98, 21)

In [37]:
# For point 3 (top sector count-wise), which company received the highest investment - D2 | Start

D2[D2.main_sector ==D2_count.sort_values(by='count', ascending=False).iloc[0,:].main_sector].groupby('company_permalink',as_index=False)['raised_amount_usd'].sum().sort_values(by='raised_amount_usd', ascending=False).head(1)

# For point 3 (top sector count-wise), which company received the highest investment - D2| End

,company_permalink,raised_amount_usd
34,/organization/electric-cloud,37000000.00


In [38]:
#For point 4 (second best sector count-wise), which company received the highest investment | Start
D2[D2.main_sector ==D2_count.sort_values(by='count', ascending=False).iloc[1,:].main_sector].groupby('company_permalink',as_index=False)['raised_amount_usd'].sum().sort_values(by='raised_amount_usd', ascending=False).head(1)
#For point 4 (second best sector count-wise), which company received the highest investment | End

,company_permalink,raised_amount_usd
30,/organization/eusa-pharma,35600000.00


In [39]:
# Total investment for D3 | Start
D3['raised_amount_usd'].sum()
# Total investment for D3 | End

2683537552.0

In [40]:
# Total investement & Total investment count for IND - D3 | Start

D3_count=D3.groupby('main_sector')['main_sector'].count().reset_index(name="count")
D3_count
D3_tot_amt = D3.groupby('main_sector')['raised_amount_usd'].sum().reset_index(name="sum")
D3_tot_amt
D3= pd.merge(pd.merge(D3,D3_count,on='main_sector'),D3_tot_amt,on='main_sector')

# Total investement & Total investment count for IND -  D3| End

In [41]:
#Top Sector name (no. of investment-wise) - D3 | Start
D3_count.sort_values(by='count', ascending=False).iloc[0,:].main_sector
#Top Sector name (no. of investment-wise) - D3 | End

'Others'

In [43]:
#Second Sector name (no. of investment-wise) - D3 | Start
D3_count.sort_values(by='count', ascending=False).iloc[1,:].main_sector
#Second Sector name (no. of investment-wise)) - D3| End

'News, Search and Messaging'

In [45]:
#Third Sector name (no. of investment-wise)- D3 | Start
D3_count.sort_values(by='count', ascending=False).iloc[2,:].main_sector
#Third Sector name (no. of investment-wise)- D3 | End

'Entertainment'

In [46]:
#Number of investments in top sector (3) - D3 | Start
D3[D3.main_sector ==D3_count.sort_values(by='count', ascending=False).iloc[0,:].main_sector].shape
#Number of investments in top sector (3) - D3 | End

(109, 21)

In [47]:
#Number of investments in second sector (4) - D3 | Start
D3[D3.main_sector ==D3_count.sort_values(by='count', ascending=False).iloc[1,:].main_sector].shape
#Number of investments in second sector (4) - D3 | End

(52, 21)

In [48]:
#Number of investments in third sector (5) - D3 | Start
D3[D3.main_sector ==D3_count.sort_values(by='count', ascending=False).iloc[2,:].main_sector].shape
#Number of investments in third sector (5) - D3 | End

(33, 21)

In [49]:
# For point 3 (top sector count-wise), which company received the highest investment - D3 | Start

D3[D3.main_sector ==D3_count.sort_values(by='count', ascending=False).iloc[0,:].main_sector].groupby('company_permalink',as_index=False)['raised_amount_usd'].sum().sort_values(by='raised_amount_usd', ascending=False).head(1)

# For point 3 (top sector count-wise), which company received the highest investment - D3| End

,company_permalink,raised_amount_usd
26,/organization/firstcry-com,39000000.00


In [50]:
#For point 4 (second best sector count-wise), which company received the highest investment - D3| Start
D3[D3.main_sector ==D3_count.sort_values(by='count', ascending=False).iloc[1,:].main_sector].groupby('company_permalink',as_index=False)['raised_amount_usd'].sum().sort_values(by='raised_amount_usd', ascending=False).head(1)
#For point 4 (second best sector count-wise), which company received the highest investment - D3 | End

,company_permalink,raised_amount_usd
10,/organization/gupshup-technology-india-pvt-ltd,33000000.00


In [51]:
D.to_csv("/Users/achintyadutta/Documents/UPGRAD/DataWarehouse - SQL/Group Asssignment/D.csv",sep=',')

In [ ]:
master_frame.to_csv('/Users/achintyadutta/Documents/UPGRAD/DataWarehouse - SQL/Group Asssignment/master_frame.csv', sep=',')

In [56]:
top9.to_csv('/Users/achintyadutta/Documents/UPGRAD/DataWarehouse - SQL/Group Asssignment/top9.csv', sep=',')